In [23]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest
from ydata_profiling import ProfileReport

In [2]:
data = pd.read_csv('~\\UNSW_NB15_training-set.csv')
print(data.shape)
data_ = pd.read_csv('~\\UNSW_NB15_testing-set.csv')
print(data_.shape)
data = pd.concat([data, data_], ignore_index=True)
print(data.shape)

(82332, 45)
(175341, 45)
(257673, 45)


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257673 entries, 0 to 257672
Data columns (total 45 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 257673 non-null  int64  
 1   dur                257673 non-null  float64
 2   proto              257673 non-null  object 
 3   service            257673 non-null  object 
 4   state              257673 non-null  object 
 5   spkts              257673 non-null  int64  
 6   dpkts              257673 non-null  int64  
 7   sbytes             257673 non-null  int64  
 8   dbytes             257673 non-null  int64  
 9   rate               257673 non-null  float64
 10  sttl               257673 non-null  int64  
 11  dttl               257673 non-null  int64  
 12  sload              257673 non-null  float64
 13  dload              257673 non-null  float64
 14  sloss              257673 non-null  int64  
 15  dloss              257673 non-null  int64  
 16  si

In [16]:
data.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


In [5]:
profile = ProfileReport(data, title="Profiling Report")

In [ ]:
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
profile.to_file("dataset_analysis.html")

In [13]:
def prepare_inputs(X):
 ohe = OneHotEncoder()
 ohe.fit(X)
 X_encoded = ohe.transform(X)
 return X_encoded

X = data[['spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
       'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin',
       'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports']]
y = data['label']

print("Shape",X.shape)
print("Shape",y.shape)

Shape (257673, 38)
Shape (257673,)


In [19]:
def featureSelection(X, y, criteria, k):
    selector = SelectKBest(criteria, k=k).fit(X,y)
    names = X.columns.values[selector.get_support()]
    scores = selector.scores_[selector.get_support()]
    names_scores = list(zip(names, scores))
    ns_df = pd.DataFrame(data = names_scores, columns=['Feature_names', 'Feature_Scores'])
    ns_df_sorted = ns_df.sort_values(['Feature_Scores', 'Feature_names'], ascending = [False, True])
    print(ns_df_sorted)
    X_transformed = selector.transform(X)
    
    X = X[[val for i,val in enumerate(X.columns) if selector.get_support()[i]]]
    return X

In [24]:
chi2 = featureSelection(X, y, chi2, 38)
chi2

        Feature_names  Feature_Scores
16              stcpb    3.405102e+13
17              dtcpb    3.321425e+13
7               sload    3.437597e+12
8               dload    2.825467e+11
4                rate    8.190298e+09
3              dbytes    1.396669e+09
2              sbytes    3.407638e+08
11             sinpkt    3.270497e+08
25  response_body_len    1.154930e+08
13               sjit    3.088187e+07
14               djit    1.629098e+07
23              dmean    1.191013e+07
5                sttl    5.856318e+06
15               swin    4.570835e+06
18               dwin    4.023445e+06
12             dinpkt    2.831710e+06
1               dpkts    1.655545e+06
10              dloss    6.358215e+05
0               spkts    4.460647e+05
31     ct_dst_src_ltm    3.436895e+05
29   ct_src_dport_ltm    3.323657e+05
30   ct_dst_sport_ltm    3.001619e+05
36         ct_srv_dst    2.051666e+05
26         ct_srv_src    1.958499e+05
35         ct_src_ltm    1.703116e+05
28         c

,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
0,2,0,496,0,90909.090200,254,0,1.803636e+08,0.000000,0,...,1,1,1,2,0,0,0,1,2,0
1,2,0,1762,0,125000.000300,254,0,8.810000e+08,0.000000,0,...,1,1,1,2,0,0,0,1,2,0
2,2,0,1068,0,200000.005100,254,0,8.544000e+08,0.000000,0,...,1,1,1,3,0,0,0,1,3,0
3,2,0,900,0,166666.660800,254,0,6.000000e+08,0.000000,0,...,2,2,1,3,0,0,0,2,3,0
4,2,0,2126,0,100000.002500,254,0,8.504000e+08,0.000000,0,...,2,2,1,3,0,0,0,2,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257668,2,0,114,0,111111.107200,254,0,5.066666e+07,0.000000,0,...,24,24,13,24,0,0,0,24,24,0
257669,10,8,620,354,33.612649,254,252,8.826286e+03,4903.492188,2,...,1,1,1,2,0,0,0,1,1,0
257670,2,0,114,0,111111.107200,254,0,5.066666e+07,0.000000,0,...,3,3,3,13,0,0,0,3,12,0
257671,2,0,114,0,111111.107200,254,0,5.066666e+07,0.000000,0,...,30,30,14,30,0,0,0,30,30,0


In [20]:
fclassi = featureSelection(X, y, f_classif, 38)
fclassi

        Feature_names  Feature_Scores
5                sttl   164379.878815
27       ct_state_ttl    75714.658058
30   ct_dst_sport_ltm    41299.776583
15               swin    39573.799812
8               dload    36481.655570
18               dwin    33493.646333
4                rate    32766.308552
29   ct_src_dport_ltm    29093.389734
31     ct_dst_src_ltm    25410.933438
23              dmean    24592.797731
16              stcpb    19712.990718
17              dtcpb    19232.264515
35         ct_src_ltm    17546.587913
36         ct_srv_dst    16859.144389
26         ct_srv_src    16683.400536
28         ct_dst_ltm    15857.338990
7               sload     7233.788219
37    is_sm_ips_ports     6780.665379
11             sinpkt     6381.042649
1               dpkts     2467.562130
10              dloss     1495.408367
3              dbytes      943.556061
14               djit      615.564368
20             synack      482.890471
0               spkts      478.216721
12          

,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
0,2,0,496,0,90909.090200,254,0,1.803636e+08,0.000000,0,...,1,1,1,2,0,0,0,1,2,0
1,2,0,1762,0,125000.000300,254,0,8.810000e+08,0.000000,0,...,1,1,1,2,0,0,0,1,2,0
2,2,0,1068,0,200000.005100,254,0,8.544000e+08,0.000000,0,...,1,1,1,3,0,0,0,1,3,0
3,2,0,900,0,166666.660800,254,0,6.000000e+08,0.000000,0,...,2,2,1,3,0,0,0,2,3,0
4,2,0,2126,0,100000.002500,254,0,8.504000e+08,0.000000,0,...,2,2,1,3,0,0,0,2,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257668,2,0,114,0,111111.107200,254,0,5.066666e+07,0.000000,0,...,24,24,13,24,0,0,0,24,24,0
257669,10,8,620,354,33.612649,254,252,8.826286e+03,4903.492188,2,...,1,1,1,2,0,0,0,1,1,0
257670,2,0,114,0,111111.107200,254,0,5.066666e+07,0.000000,0,...,3,3,3,13,0,0,0,3,12,0
257671,2,0,114,0,111111.107200,254,0,5.066666e+07,0.000000,0,...,30,30,14,30,0,0,0,30,30,0


In [22]:
mutual_classi = featureSelection(X, y, mutual_info_classif, 38)
mutual_classi

        Feature_names  Feature_Scores
2              sbytes        0.455866
27       ct_state_ttl        0.340200
7               sload        0.340082
3              dbytes        0.338613
5                sttl        0.338515
22              smean        0.334958
6                dttl        0.316088
4                rate        0.313244
23              dmean        0.283733
12             dinpkt        0.270835
8               dload        0.252349
1               dpkts        0.242424
11             sinpkt        0.226328
19             tcprtt        0.218932
20             synack        0.217440
21             ackdat        0.209057
13               sjit        0.185478
14               djit        0.161925
0               spkts        0.159649
30   ct_dst_sport_ltm        0.155785
10              dloss        0.126483
9               sloss        0.108655
36         ct_srv_dst        0.098691
29   ct_src_dport_ltm        0.095092
15               swin        0.087162
28         c

,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
0,2,0,496,0,90909.090200,254,0,1.803636e+08,0.000000,0,...,1,1,1,2,0,0,0,1,2,0
1,2,0,1762,0,125000.000300,254,0,8.810000e+08,0.000000,0,...,1,1,1,2,0,0,0,1,2,0
2,2,0,1068,0,200000.005100,254,0,8.544000e+08,0.000000,0,...,1,1,1,3,0,0,0,1,3,0
3,2,0,900,0,166666.660800,254,0,6.000000e+08,0.000000,0,...,2,2,1,3,0,0,0,2,3,0
4,2,0,2126,0,100000.002500,254,0,8.504000e+08,0.000000,0,...,2,2,1,3,0,0,0,2,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257668,2,0,114,0,111111.107200,254,0,5.066666e+07,0.000000,0,...,24,24,13,24,0,0,0,24,24,0
257669,10,8,620,354,33.612649,254,252,8.826286e+03,4903.492188,2,...,1,1,1,2,0,0,0,1,1,0
257670,2,0,114,0,111111.107200,254,0,5.066666e+07,0.000000,0,...,3,3,3,13,0,0,0,3,12,0
257671,2,0,114,0,111111.107200,254,0,5.066666e+07,0.000000,0,...,30,30,14,30,0,0,0,30,30,0
